# Electric mix
The production of electrical energy from renewable sources has become a crucial issue in the face of current climate challenges. Electricity distribution networks now integrate numerous renewable energy sources, and the ultimate goal is to achieve a fully renewable electrical production, relying on a diversified energy mix including photovoltaic, wind, hydroelectric, biomass, geothermal, and many others. Each means of production has associated costs, which are not limited to financial aspects but also encompass environmental considerations. Thus, there are different ways to combine these energy sources. 

In this perspective, we propose a study focusing on the use of quantum computing to find an optimal combination of renewable energies while minimizing costs. However, it is important to emphasize that the work presented in this study is only a demonstration and does not rely on real data or a concrete model. Its objective is to highlight the potential resolution of this problem through quantum computing. In this regard, we strive to find, using a quantum approach, an energy combination that minimizes costs.
   
 # Problem
 
 More concretely, the problem can be stated as follows:
 
*Given a city with an energy requirement X and having n different sources of energy production (all assumed to be renewable), what is the optimal combination such that the overall cost of electricity production for this city is minimized?*

The problem we are formulating is equivalent to other well-known problems such as the knapsack problem or the traveling salesman problem. These problems can be interpreted as quadratic problems because the cost can be modeled by a quadratic function subject to certain constraints. In our case, these constraints may include the maximum energy capacity of each source and the minimum energy requirement.

To build our cost function, we can first consider that it should be the sum of the costs of each involved energy source. Then, it should take into account the mutual influence of the energy sources. This leads us to a simplified model that can be expressed mathematically as follows:

We will only consider hydroelectric (h), solar (s), and wind (e) energy sources.

 Cost function: $f(h , s , e) = h + s + e + \frac{1}{3}h.s + \frac{1}{3}h.e +\frac{1}{3}e.s$

Constraints :
- $0 \leqslant h \leqslant 4$
- $0 \leqslant s \leqslant 3$
- $0 \leqslant e \leqslant 2$
- $ h + s + e \geqslant 2$

# Implementation

To solve this problem, we will implement two quantum optimization algorithms:

- The **Quantum Approximate Optimization Algorithm (QAOA)** utilizes variable parameters to create a quantum superposition to efficiently explore the search space. Measurements are then performed to obtain an approximate solution to the optimization problem at hand.

- The **Variational Quantum Eigensolver (VQE)** algorithm is used to estimate eigenvalues and eigenvectors of a quantum system. It relies on variational and optimization techniques to find an approximation of the ground states of a given quantum system. By using parameterized quantum circuits, VQE attempts to minimize the expected energy of the quantum system by adjusting the circuit parameters. This allows for an estimation of the desired quantum state, even on limited-scale quantum computers.

Finally, we will compare our results with a classical approach.

## Initialization

In [1]:
from qiskit import Aer
from qiskit.algorithms import QAOA, VQE , NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

from qiskit.utils import QuantumInstance , algorithm_globals
from qiskit.providers.aer.noise.noise_model import NoiseModel


#Optimization package
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit_optimization.converters import QuadraticProgramToQubo

import qiskit.test.mock as fake

## Mapping the problem

In [2]:
mixElectric = QuadraticProgram(name = "electricity mix")
mixElectric.integer_var(name = "s" , lowerbound = 0 , upperbound = 3)
mixElectric.integer_var(name = "h" , lowerbound = 0 , upperbound = 4)
mixElectric.integer_var(name = "e" , lowerbound = 0 , upperbound = 2)

mixElectric.minimize(
linear = {"s":2 , "h":3 , "e":1} ,
quadratic = {("s","h"):(1/3) , ("s","e"):(1/3) , ("h","e"):(1/3)})

mixElectric.linear_constraint(linear = {"s":1 , "h":1 , "e":1} , sense= ">=" , rhs = 7)

print(mixElectric.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: electricity mix

Minimize
 obj: 2 s + 3 h + _e + [ 0.666666666667 s*h + 0.666666666667 s*_e
      + 0.666666666667 h*_e ]/2
Subject To
 c0: s + h + _e >= 7

Bounds
       s <= 3
       h <= 4
       _e <= 2

Generals
 s h _e
End



To execute and apply our algorithms to our problem, we will transform it into binary and convert it from a quadratic problem to an Unconstrained Binary Optimization problem (QUBO).


In [3]:
qubo_version , _ = (QuadraticProgramToQubo().convert(mixElectric, ).to_ising())

print(f"The required number of qubits is:{qubo_version.num_qubits}")

The required number of qubits is:9


In [4]:
QuadraticProgramToQubo().convert(mixElectric)

<QuadraticProgram: minimize 29.666666666666664*c0@int_slack@0^2 + 59.333333333..., 9 variables, 0 constraints, 'electricity mix'>

# QAOA Solution

In [5]:
def QAOA_solution(
mixElectric: QuadraticProgram, quantumInstance:QuantumInstance, optimizer = None,):
    _eval_count = 0
    
    def callback(eval_count , parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count
        
    solver = QAOA(
        optimizer = optimizer, quantum_instance = quantumInstance, callback = callback)
    optimizer = MinimumEigenOptimizer(solver)
        
    result = optimizer.solve(mixElectric)
    return result, _eval_count

In [6]:
simulator_instance = QuantumInstance(
backend = Aer.get_backend("qasm_simulator"),
seed_simulator = algorithm_globals.random_seed,
seed_transpiler = algorithm_globals.random_seed,
)

#result

qaoa_result, qaoa_eval_count = QAOA_solution(mixElectric, simulator_instance)

# Format and print result
print("Solution with QAOA method:\n")
print(f"minimum cost is {qaoa_result.fval}")
print(f"energies: ")
for energy_quantity, energy_name in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{(energy_quantity/7)*100} percent of {energy_name}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

C:\Users\isaac\AppData\Local\Temp\ipykernel_43436\2624796069.py:1: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  simulator_instance = QuantumInstance(
C:\Users\isaac\AppData\Local\Temp\ipykernel_43436\3576925875.py:9: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  solver = QAOA(


Solution with QAOA method:

minimum cost is 19.333333333333332
energies: 
	42.857142857142854 percent of s
	28.57142857142857 percent of h
	28.57142857142857 percent of e

The solution was found within 196 evaluations of QAOA.


# VQE solution

In [7]:
def VQE_solution(
mixElectric: QuadraticProgram, quantumInstance:QuantumInstance, optimizer = None,):
    _eval_count = 0
    
    def callback(eval_count , parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count
        
    solver = VQE(
        optimizer = optimizer, quantum_instance = quantumInstance, callback = callback)
    optimizer = MinimumEigenOptimizer(solver)
        
    result = optimizer.solve(mixElectric)
    return result, _eval_count

In [8]:
#create a QuantumInstance
simulator_instance = QuantumInstance(
    backend = Aer.get_backend("qasm_simulator"),
    seed_simulator = algorithm_globals.random_seed,
    seed_transpiler = algorithm_globals.random_seed,
)

#Get VQE result

vqe_result, vqe_eval_count = VQE_solution(mixElectric , simulator_instance)

# Format and print result
print("Solution found using the VQE method:\n")
print(f"minimun cost is{vqe_result.fval}")
print(f"energy used are: ")
for energy_quantity, energy_name in zip(vqe_result.x, vqe_result.variable_names):
    print(f"\t{(energy_quantity/7)*100} percent of {energy_name}")

print(f"\nThe solution was found within {vqe_eval_count} evaluations of VQE")


C:\Users\isaac\AppData\Local\Temp\ipykernel_43436\375443605.py:2: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  simulator_instance = QuantumInstance(
C:\Users\isaac\AppData\Local\Temp\ipykernel_43436\2172370032.py:9: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  solver = VQE(


Solution found using the VQE method:

minimun cost is19.333333333333332
energy used are: 
	42.857142857142854 percent of s
	28.57142857142857 percent of h
	28.57142857142857 percent of e

The solution was found within 37 evaluations of VQE


# Classical solution

In [9]:
def classical_solution(mixElectric: QuadraticProgram):
    solver = NumPyMinimumEigensolver()
    optimizer = MinimumEigenOptimizer(solver)
    
    return optimizer.solve(mixElectric)
    

In [10]:
classical_result = classical_solution(mixElectric)

print("Solution found using the classical method:\n")
print(f"minimum cost is {classical_result.fval} ")
print(f"les sources d'énérgies sont: ")

_energy = [v.name for v in mixElectric.variables]
for Index, energy in enumerate(classical_result.x):
    print(f"\t{(energy/7)*100} percent of {_energy[Index]}")

Solution found using the classical method:

minimum cost is 19.333333333333332 
les sources d'énérgies sont: 
	42.857142857142854 percent of s
	28.57142857142857 percent of h
	28.57142857142857 percent of e


# Discussion and conclusion

As can be seen, the three approaches yield the same results. The VQE algorithm performs better, requiring only 37 evaluations, while the QAOA algorithm requires 183.

Although the model is simple and likely disconnected from reality, it provides at least an overview of the reliability of a quantum approach. With a sufficiently large database on the deployment of electrical energy for a given area, it would be possible to build a realistic model.

Even though we used a local simulation for this work, there is currently no issue in using a real computer.

In [11]:
import qiskit.tools.jupyter

%qiskit_version_table